In [1]:
from datetime import datetime, timedelta, date, time
import pandas as pd
import numpy as np
from basic.claims import connect, get_users, trans
import matplotlib.pyplot as plt
import seaborn as sns
from bson import ObjectId
import requests
from time import sleep
import json
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
client, es = connect()

Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [3]:
def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

def vbrr_stavki(payout, partner):
    if partner=='МигКредит':
        if payout==3330:
            return 2750.00
        elif payout==8000:
            return 6666.00
        elif payout==9000:
            return 7500.00
        elif payout==10000:
            return 8334.00
        elif payout==2000:
            return 1600.00
        elif payout==450:
            return 375.00
        else:
            return round(payout, 2)
    elif partner=='MoneyMan':
        return 2710.00
    else:
        return round(payout/1.2, 2)

def get_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()),
                          "$lt": datetime.combine(till, datetime.min.time())},
           'product.productType': 'LOAN'
#           "whence": {"$in": whence_list}
          }
    results_count = client['claim-service']['claims'].count_documents(req)
    print(results_count)
    all_ans = []
    batch_size = 1000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, 
                                                          {"fieldValues": 0} ).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    ans = trans(all_ans, client)
    df = pd.DataFrame(ans)

    return df

# def get_claims2(since, till, client):
#     req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()),
#                           "$lt": datetime.combine(till, datetime.min.time())},
#           "$or": [{"whence": {"$regex": "t2sms"}}, {"whence": {"$regex": "abrakadabra"}},]
#           }
#     results_count = client['claim-service']['claims'].count_documents(req)
#     print(results_count)
#     all_ans = []
#     batch_size = 1000
#     for i in range(results_count//batch_size+1):
#         ans = list(client['claim-service']['claims'].find(req, 
#                                                           {"fieldValues": 0} ).limit(batch_size).skip(i*batch_size))
#         all_ans = all_ans + ans
#     ans = trans(all_ans, client)
#     df = pd.DataFrame(ans)

#     return df

def get_claims_Id(client, claim_list):
    req = {"_id": {"$in": [ObjectId(ci) for ci in claim_list]}
          }
    results_count = client['claim-service']['claims'].count_documents(req)
    print(results_count)
    all_ans = []
    batch_size = 1000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, 
                                                          {"fieldValues": 0} ).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    ans = trans(all_ans, client)
    df = pd.DataFrame(ans)

    return df

def cre_get(es, since):
    
    chunk = 5000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", ascending=False).drop_duplicates('userId', keep='first')
            
    return df[['userId', 'score']].reset_index(drop=True)

def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW1"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df

def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

till = date.today()+timedelta(days=1)
# till = date(2021,6,)
since = date(2021,9,1)
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dff_all = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

# dff_all1 = get_claims(since, till, client, whence_list)
# dff_all = get_claims(since, till, client)
# # dff_all = pd.concat([dff_all1, dff_all2])
# dff_all = pd.merge(dff_all, dp, left_on='id', 
#                    right_on='transitionId', how='left')
dff_all['payout'] = dff_all['payout'].apply(lambda x: x.replace('{sum}', '0') if type(x)==str else x).astype(float)
# dff_all['payment'] = dff_all.apply(lambda x: vbrr_stavki(x['payout'], x['partner']), axis=1)

dff_all['status_x'] = dff_all.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)

55027
100241; 90241; 80241; 70241; 60241; 50241; 40241; 30241; 20241; 10241; 241; 38668; 28668; 18668; 8668; 

In [4]:
# dff_all['partner'] = dff_all.apply(lambda x: 'MoneyMan API' if
#                         x['partner']=='MoneyMan' and x['type']=='API' 
#                           else x['partner'], axis=1)
# dff_all['partner'] = dff_all.apply(lambda x: 'MoneyMan Ракета Деньги' if
#                         x['partner']=='MoneyMan' or x['partner']=='MoneyMan API' and x['product_name']=='Ракета Деньги' 
#                           else x['partner'], axis=1)

In [5]:
dff_all['week'] = (pd.to_datetime(dff_all['dt']).dt.week)+1
dff_all['week'] = dff_all['week'].astype('int')


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [6]:
dff_all['partner'] = dff_all.apply(lambda x: 'MoneyMan API' if
                        x['partner']=='MoneyMan' and x['type']=='API' 
                          else x['partner'], axis=1)
dff_all['partner'] = dff_all.apply(lambda x: 'MoneyMan Ракета Деньги' if
                         x['product_name']=='Ракета Деньги' 
                          else x['partner'], axis=1)
dff_all['partner'] = dff_all.apply(lambda x: 'MoneyMan Рефинансирование' if
                         x['product_name']=='Рефинансирование займов от Moneyman' 
                          else x['partner'], axis=1)

In [7]:
# group=dff_all[dff_all['week']>=27].groupby(['borrower_userId','adapterName','product_name',
#                        'status',
#                        'week','dt']).agg({'id':'count'}).sort_values('id',ascending=False)
# group.head(1000)

In [8]:
# group.boxplot('id')

In [9]:
# # temporary cell
# cre = cre_get(es, since)
# dff_all = pd.merge(dff_all, cre, left_on='borrower_userId', 
#                    right_on='userId', how='left')

In [10]:
# dff_all['group'] = dff_all['score'].apply(lambda x: '1-24' if pd.notnull(x) and x<25 and x>1 else 
#                                  '25-49' if pd.notnull(x) and x<50 and x>1 else 
#                                  '50-74' if pd.notnull(x) and x<75 and x>1 else 
#                                  '75-99' if pd.notnull(x) and x<100 and x>1 else np.nan)
# dff_all[dff_all['score'].notnull()][['score', 'group']]

In [11]:
dff_all = dff_all[dff_all['product_type']=='LOAN']

In [12]:
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' or x[:2]=='t2' else x)
# dff_all['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
#                                    x in ['tele2TommorowWON',
#                                          'tele2Tommorow',
#                                          'tele2FourDays',
#                                          'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dff_all['whence_x'] =dff_all['whence_x'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele'  else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Теле2 \\ Кредитные карты' if 
                                   x[:5]=='t2_kk' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dff_all['whence_x'] = dff_all['whence_x'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)

In [13]:
#dff_all['rank'] = 

In [14]:
# dff_all['c'] = dff_all.sort_values('createdAt', ascending=True).groupby(['product_id',
#                                                                     'whence_x']).cumcount('rank')

In [15]:
fields = ["week","partner", "product_name","whence_x",]
c = dff_all.groupby(fields).agg({'userId_x': 'nunique'})
z = dff_all[dff_all['status_x']!='NEW'].groupby(fields).agg({'userId_x': 'nunique'})
v = dff_all[dff_all['status_x'].isin(['ISSUED', "APPROVED"])].groupby(fields).agg({"userId_x": "count",
                                                                                           "payout": "sum"})

cz = pd.merge(left=c, right=z, left_index=True, right_index=True, how='left')
czv = pd.merge(left=cz, right=v, left_index=True, right_index=True, how='left')
czv.columns = ['Клики', 'Заявки', 'Выдачи', 'Получено', ]
czv = czv.fillna(0).astype(int).reset_index()

In [16]:
czv.head(1000)

,week,partner,product_name,whence_x,Клики,Заявки,Выдачи,Получено
0,36,Ryabina credit,Ryabina credit,Тестирование гипотез,2,1,1,75
1,36,Vivus,Vivus,Admitad,2,0,0,0
2,36,Vivus,Vivus,Cpahub,1,0,0,0
3,36,Vivus,Vivus,SMS рекламные,5,2,0,0
4,36,Vivus,Vivus,Tele2 // Микрозаймы,1,0,0,0
5,36,Vivus,Vivus,Ссылочная масса,2,0,0,0
6,36,Capitan.Money,Capitan.Money,SMS рекламные,7,2,2,200
7,36,Cash-U,Cash-U заем до 5000 руб. под 0%*,Email,1,0,0,0
8,36,Cash-U,Cash-U заем до 5000 руб. под 0%*,Tele2 // Микрозаймы,2,0,0,0
9,36,Cash-U,Cash-U заем до 5000 руб. под 0%*,Ссылочная масса,1,1,1,2800


In [17]:
c1 = dff_all.groupby(fields).agg({"userId": "count"})
c1

userId
week partner         product_name        whence_x                    
36    Ryabina credit Ryabina credit      Тестирование гипотез       0
      Vivus          Vivus               Admitad                    0
                                         Cpahub                     0
                                         SMS рекламные              0
                                         Tele2 // Микрозаймы        0
...                                                               ...
52   Умные Наличные  Выдача займа онлайн Ссылочная масса            1
     Финтерра        Финтерра            Email                      0
                                         Ссылочная масса            2
     еКапуста        еКапуста            Tele2 // Микрозаймы        0
                                         Ссылочная масса            1

[2294 rows x 1 columns]

In [18]:
czv

,week,partner,product_name,whence_x,Клики,Заявки,Выдачи,Получено
0,36,Ryabina credit,Ryabina credit,Тестирование гипотез,2,1,1,75
1,36,Vivus,Vivus,Admitad,2,0,0,0
2,36,Vivus,Vivus,Cpahub,1,0,0,0
3,36,Vivus,Vivus,SMS рекламные,5,2,0,0
4,36,Vivus,Vivus,Tele2 // Микрозаймы,1,0,0,0
...,...,...,...,...,...,...,...,...
2289,52,Умные Наличные,Выдача займа онлайн,Ссылочная масса,2,0,0,0
2290,52,Финтерра,Финтерра,Email,2,1,0,0
2291,52,Финтерра,Финтерра,Ссылочная масса,2,0,0,0
2292,52,еКапуста,еКапуста,Tele2 // Микрозаймы,3,1,0,0


In [19]:
czv['CR'] = ((czv['Заявки']/czv['Клики'])*100).round(2)
czv['AR'] = ((czv['Выдачи']/czv['Заявки'])*100).round(2)
czv['EPC'] = (czv['Получено']/czv['Клики']).round(2)
czv['EPL'] = (czv['Получено']/czv['Заявки']).round(2)

In [20]:
#czv = czv.fillna(0)
#czv.replace([np.inf, -np.inf], np.nan, inplace=True)


In [21]:
#czv = czv[czv['Клики']>=20]
#czv = czv[czv['whence']!='__']

In [22]:
# czv = czv[czv['week'].isin([45,44,43,42])]
czv

,week,partner,product_name,whence_x,Клики,Заявки,Выдачи,Получено,CR,AR,EPC,EPL
0,36,Ryabina credit,Ryabina credit,Тестирование гипотез,2,1,1,75,50.00,100.0,37.5,75.0
1,36,Vivus,Vivus,Admitad,2,0,0,0,0.00,NaN,0.0,NaN
2,36,Vivus,Vivus,Cpahub,1,0,0,0,0.00,NaN,0.0,NaN
3,36,Vivus,Vivus,SMS рекламные,5,2,0,0,40.00,0.0,0.0,0.0
4,36,Vivus,Vivus,Tele2 // Микрозаймы,1,0,0,0,0.00,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2289,52,Умные Наличные,Выдача займа онлайн,Ссылочная масса,2,0,0,0,0.00,NaN,0.0,NaN
2290,52,Финтерра,Финтерра,Email,2,1,0,0,50.00,0.0,0.0,0.0
2291,52,Финтерра,Финтерра,Ссылочная масса,2,0,0,0,0.00,NaN,0.0,NaN
2292,52,еКапуста,еКапуста,Tele2 // Микрозаймы,3,1,0,0,33.33,0.0,0.0,0.0


In [23]:
czv_epc = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='EPC').fillna(0)
czv_epc=czv_epc.sort_index(axis='columns', level='week',ascending=False)
czv_epc['sum'] = czv_epc.sum(axis=1)
#czv_epc = czv_epc[czv_epc['sum']>0]
czv_epc=czv_epc.reset_index()

In [24]:
czv_epc

week,partner,product_name,whence_x,52,51,50,49,48,47,46,45,44,43,42,41,40,39,38,37,36,sum
0,Ryabina credit,Ryabina credit,__,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Ryabina credit,Ryabina credit,Тестирование гипотез,0.0,0.00,0.00,0.00,0.00,0.00,0.00,75.00,12.50,0.00,50.00,18.75,50.00,0.00,0.00,0.00,37.50,243.75
2,Vivus,Vivus,Admitad,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Vivus,Vivus,Cpahub,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,126.00,0.00,0.00,126.00
4,Vivus,Vivus,Email,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Vivus,Vivus,SMS рекламные,0.0,0.00,0.00,210.00,28.37,0.00,0.00,0.00,295.71,50.32,0.00,0.00,83.57,0.00,0.00,0.00,0.00,667.97
6,Vivus,Vivus,Tele2 // Микрозаймы,0.0,0.00,0.00,0.00,45.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,45.00
7,Vivus,Vivus,__,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,Vivus,Vivus,guruleads_0443ce9551e3d126a49d61c3139fe8ce_9155,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Vivus,Vivus,lostBasket,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [25]:
czv_epc.to_excel('epcweek/epc.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/epc.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [26]:
czv_epl = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='EPL').fillna(0)
czv_epl=czv_epl.sort_index(axis='columns', level='week',ascending=False)
czv_epl['sum'] = czv_epl.sum(axis=1)
czv_epl =czv_epl.reset_index()
czv_epl.to_excel('epcweek/epl.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/epl.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [27]:
czv_cr = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='CR').fillna(0)
czv_cr=czv_cr.sort_index(axis='columns', level='week',ascending=False)
czv_cr['sum'] = czv_cr.sum(axis=1)
czv_cr = czv_cr.reset_index()
czv_cr.to_excel('epcweek/cr.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/cr.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [28]:
czv_ar = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='AR').fillna(0)
czv_ar=czv_ar.sort_index(axis='columns', level='week',ascending=False)
czv_ar['sum'] = czv_ar.sum(axis=1)
czv_ar = czv_ar.reset_index()
czv_ar.to_excel('epcweek/ar.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/ar.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [29]:
czv_click = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='Клики').fillna(0)
czv_click=czv_click.sort_index(axis='columns', level='week',ascending=False)
czv_click['sum'] = czv_click.sum(axis=1)
czv_click = czv_click.reset_index()
czv_click.to_excel('epcweek/czv_click.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/czv_click.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [30]:
czv_zayavki = czv.pivot_table(index=['partner','product_name','whence_x'],columns='week',values='Заявки').fillna(0)
czv_zayavki=czv_zayavki.sort_index(axis='columns', level='week',ascending=False)
czv_zayavki['sum'] = czv_zayavki.sum(axis=1)
czv_zayavki = czv_zayavki.reset_index()
czv_zayavki.to_excel('epcweek/czv_zayavki.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/czv_zayavki.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [31]:
filename = f'reports/epc_{date.today()}.xlsx'
czv.sort_values('EPC', ascending=False).to_excel(filename)

In [32]:
# with open(filename, 'rb') as file_:
# #     file_
#     data = {'chat_id': -427853392,
#             'document': file_,
# #             'text': '1',
# #             'parse_mode': 'markdown'
# #            'filename': f'reports/cr_pl_{date.today()-timedelta(days=1)}.xlsx'
#            }
#     rf = requests.post('https://api.telegram.org/bot709510447:AAH4hlOCNhlDWXGs3bWtotej8EDDn1ojJ4w/sendDocument?chat_id=-427853392', 
#                        files=data)
# print(rf.status_code, rf.reason, rf.content)

In [33]:
fields = ["week","partner", "product_name"]
c1 = dff_all[dff_all['week']==51].groupby(fields).agg({"userId_x": "nunique"})
z1 = dff_all[dff_all['status_x']!='NEW'].groupby(fields).agg({"userId_x": "nunique"})
v1 = dff_all[dff_all['status_x'].isin(['ISSUED', "APPROVED"])].groupby(fields).agg({"userId_x": "nunique",
                                                                                           "payout": "sum"})

cz = pd.merge(left=c1, right=z1, left_index=True, right_index=True, how='left')
czv1 = pd.merge(left=cz, right=v1, left_index=True, right_index=True, how='left')
czv1.columns = ['Клики', 'Заявки', 'Выдачи', 'Получено', ]
czv1 = czv1.fillna(0).astype(int).reset_index()

czv1 = czv1.sort_values('Клики',ascending=False)

czv1['EPC'] = (czv1['Получено']/czv1['Клики']).round(2)
czv1['EPL'] = (czv1['Получено']/czv1['Заявки']).round(2)

czv1 = czv1.fillna(0)

In [34]:
czv1

,week,partner,product_name,Клики,Заявки,Выдачи,Получено,EPC,EPL
30,51,Доброзайм,Доброзайм,114,15,3,6768,59.37,451.2
9,51,Konga,Konga,65,8,0,0,0.00,0.0
42,51,Финтерра,Финтерра,63,8,1,800,12.70,100.0
25,51,Быстроденьги,Быстроденьги онлайн АКЦИЯ,54,0,0,0,0.00,0.0
29,51,До Зарплаты,До Зарплаты,52,6,2,5952,114.46,992.0
44,51,еКапуста,еКапуста,38,5,1,2325,61.18,465.0
31,51,Доброзайм,Доступный займ,27,3,1,360,13.33,120.0
0,51,Vivus,Vivus,27,4,0,0,0.00,0.0
39,51,Срочноденьги,Срочноденьги,26,6,0,0,0.00,0.0
34,51,Займер,Займер,26,5,2,1600,61.54,320.0


In [35]:
czv1.to_excel('epcweek/czv_svod.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/czv_svod.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [36]:
fields = ["week","partner", "product_name"]
c1 = dff_all[dff_all['week']==52].groupby(fields).agg({"userId_x": "nunique"})
z1 = dff_all[dff_all['status_x']!='NEW'].groupby(fields).agg({"userId_x": "nunique"})
v1 = dff_all[dff_all['status_x'].isin(['ISSUED', "APPROVED"])].groupby(fields).agg({"userId_x": "nunique",
                                                                                           "payout": "sum"})

cz = pd.merge(left=c1, right=z1, left_index=True, right_index=True, how='left')
czv1 = pd.merge(left=cz, right=v1, left_index=True, right_index=True, how='left')
czv1.columns = ['Клики', 'Заявки', 'Выдачи', 'Получено', ]
czv1 = czv1.fillna(0).astype(int).reset_index()

czv1 = czv1.sort_values('Клики',ascending=False)

czv1['EPC'] = (czv1['Получено']/czv1['Клики']).round(2)
czv1['EPL'] = (czv1['Получено']/czv1['Заявки']).round(2)

czv1 = czv1.fillna(0)

In [37]:
czv1.to_excel('epcweek/czv_svod1.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1OgPREms_rRaYwnjjdNmJ7Hjfi1LzVGXsrcuKcRcfR6I')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('epcweek/czv_svod1.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [38]:
czv1

,week,partner,product_name,Клики,Заявки,Выдачи,Получено,EPC,EPL
5,52,Konga,Konga,5,0,0,0,0.0,0.0
20,52,Доброзайм,Доброзайм,5,0,0,0,0.0,0.0
0,52,Vivus,Vivus,4,0,0,0,0.0,0.0
31,52,Финтерра,Финтерра,4,1,0,0,0.0,0.0
15,52,Zaimon,Zaimon,4,0,0,0,0.0,0.0
16,52,Zaymigo,Zaymigo,4,0,0,0,0.0,0.0
32,52,еКапуста,еКапуста,4,1,0,0,0.0,0.0
12,52,Platiza,Platiza,3,0,0,0,0.0,0.0
24,52,Займер,Займер,3,0,0,0,0.0,0.0
17,52,Быстроденьги,Быстроденьги онлайн АКЦИЯ,3,0,0,0,0.0,0.0
